<a href="https://colab.research.google.com/github/subhasisj/python-projects/blob/master/Machine%20Learning%20Projects/PySpark%20ML/sentiment_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:12 https://developer.download.nvidia.com/compute/machine-lear

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [6]:
data = spark.read.csv('clean_tweet.csv',header=True)
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- text: string (nullable = true)
 |-- target: string (nullable = true)



In [7]:
data.show(5)

+---+--------------------+------+
|_c0|                text|target|
+---+--------------------+------+
|  0|awww that s a bum...|     0|
|  1|is upset that he ...|     0|
|  2|i dived many time...|     0|
|  3|my whole body fee...|     0|
|  4|no it s not behav...|     0|
+---+--------------------+------+
only showing top 5 rows



In [8]:
data = data.dropna()
data.count()

56631

In [0]:
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.types import StructType,StructField,IntegerType
from pyspark.ml.feature import CountVectorizer,HashingTF,IDF,Tokenizer,StringIndexer,Imputer

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression,NaiveBayes


In [0]:
( train_set ,test_set ,validation_set ) = data.randomSplit([0.8,0.15,0.05],seed = 4)

In [24]:
tokenizer = Tokenizer(inputCol = 'text',outputCol = 'words')

hashtf = HashingTF(numFeatures=2**16,inputCol = 'words',outputCol = 'tf')
# cv = CountVectorizer(vocabSize=2**16, inputCol="words", outputCol='cv')

idf = IDF(inputCol = 'tf', outputCol = 'features' ,minDocFreq =5)

label_string_indexer = StringIndexer(inputCol = 'target',outputCol= 'label' )

# log_reg = LogisticRegression(maxIter=100)
# nb = NaiveBayes()
pipeline = Pipeline(stages = [tokenizer,hashtf,idf,label_string_indexer])

pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
validation_set_df = pipelineFit.transform(validation_set)
train_df.show(5)

+----+--------------------+------+--------------------+--------------------+--------------------+-----+
| _c0|                text|target|               words|                  tf|            features|label|
+----+--------------------+------+--------------------+--------------------+--------------------+-----+
|   0|awww that s a bum...|     0|[awww, that, s, a...|(65536,[8436,8847...|(65536,[8436,8847...|  0.0|
|   1|is upset that he ...|     0|[is, upset, that,...|(65536,[1444,2071...|(65536,[1444,2071...|  0.0|
|  10|spring break in p...|     0|[spring, break, i...|(65536,[20639,207...|(65536,[20639,207...|  0.0|
| 100|body of missing n...|     0|[body, of, missin...|(65536,[9639,1302...|(65536,[9639,1302...|  0.0|
|1000|tulip um that wou...|     0|[tulip, um, that,...|(65536,[3331,8436...|(65536,[3331,8436...|  0.0|
+----+--------------------+------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows



In [0]:
log_reg = LogisticRegression(maxIter=100)
log_reg_model=log_reg.fit(train_df)
preds = log_reg_model.transform(validation_set_df)

In [26]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(preds)

0.0

In [27]:
accuracy = preds.filter(preds.label == preds.prediction).count() / float(validation_set.count())
accuracy

1.0

In [28]:
evaluator.getMetricName()


'areaUnderROC'

In [29]:
validation_set_df.select(validation_set_df.label).filter('label == 1.0').show()

+-----+
|label|
+-----+
+-----+

